<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/week14/14_consensus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

In [2]:
!go version

go version go1.10.4 linux/amd64


In [31]:
%%writefile nodo.go
package main

import (
    "encoding/json"
    "fmt"
    "math/rand"
    "net"
    "os"
    "time"
)

type Msg struct {
    Command     string
    Hostname    string
    List        []string
}

var friends         []string
var local           string
var end             chan bool

var ready2listen    chan bool
var decisions       map[string]string
var cont            int

func serv() {
    // fmt.Println("(", local, ")")
    ln, _ := net.Listen("tcp", local)
    defer ln.Close()
    for {
        conn, _ := ln.Accept()
        go handle(conn)
    }
}
func handle(conn net.Conn) {
    defer conn.Close()
    dec := json.NewDecoder(conn)
    var msg Msg
    if err := dec.Decode(&msg); err == nil {
        switch msg.Command {
        case "hello":
            resp := Msg{"hey", local, friends}
            enc := json.NewEncoder(conn)
            if err := enc.Encode(&resp); err == nil {
                for _, friend := range friends {
                    // fmt.Println(local, friend, "meet", msg.Hostname)
                    send(friend, "meet new friend", []string{msg.Hostname})
                }
            }
            friends = append(friends, msg.Hostname)
            // fmt.Println(local, "updated list", friends)
        case "meet new friend":
            friends = append(friends, msg.List...)
            // fmt.Println(local, "new friend", msg.List)

        case "test consensus":
            if rand.Intn(100) % 2 == 0 {
                decisions[local] = "atacar"
            } else {
                decisions[local] = "retirada"
            }
            fmt.Println(local, decisions[local])
            cont = 0
            for _, friend := range friends {
                send(friend, "decision", []string{decisions[local]})
            }
            ready2listen<-true
        case "decision":
            <-ready2listen
            decisions[msg.Hostname] = msg.List[0]
            cont++
            if cont == len(friends) {
                contAtack := 0
                contFallb := 0
                for _, decision := range decisions {
                    if decision == "atacar" {
                        contAtack++
                    } else {
                        contFallb++
                    }
                }
                if contAtack < contFallb {
                    fmt.Println(local, "RETIRADA!!")
                } else {
                    fmt.Println(local, "ATACAR!!!!")
                }
                end<-true
            } else {
                ready2listen<-true
            }

        case "finish":
            end<-true
        }
    }
}
func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, local, list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        // fmt.Println(local, "sent", msg)
        if command == "hello" {
            dec := json.NewDecoder(conn)
            var resp Msg
            if err := dec.Decode(&resp); err == nil {
                friends = append(friends, resp.List...)
                // fmt.Println(local, "recibí", resp.List)
            }
        }
    }
}
func main() {
    ready2listen    = make(chan bool)
    end             = make(chan bool)
    local           = os.Args[1]
    decisions       = make(map[string]string)

    rand.Seed(time.Now().UTC().UnixNano())
    go serv()

    if len(os.Args) == 3 {
        remote := os.Args[2]
        friends = append(friends, os.Args[2])
        send(remote, "hello", []string{})
    }

    <-end
    fmt.Println(local, "time to die")
    // fmt.Println(local, friends)
}

Overwriting nodo.go


In [32]:
!go build nodo.go

In [22]:
%%writefile consensus.go
package main

import (
    "encoding/json"
    "fmt"
    "net"
    "os"
)

type Msg struct {
    Command string
    Hostname string
    List []string
}

func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, "", list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        fmt.Println("sending test consensus to", remote)
    }
}

func main() {
    for _, remote := range os.Args[1:] {
        send(remote, "test consensus", []string{})
    }
}

Overwriting consensus.go


In [23]:
!go build consensus.go

In [34]:
%%script bash

./nodo localhost:8000 &
sleep 1
./nodo localhost:8001 localhost:8000 &
sleep 1
./nodo localhost:8002 localhost:8000 &
sleep 1
./nodo localhost:8003 localhost:8002 &
sleep 1
./nodo localhost:8004 localhost:8001 &
sleep 1
./nodo localhost:8005 localhost:8003 &
sleep 1
./nodo localhost:8006 localhost:8004 &
sleep 1

./consensus localhost:8000 \
            localhost:8001 \
            localhost:8002 \
            localhost:8003 \
            localhost:8004 \
            localhost:8005 \
            localhost:8006


sending test consensus to localhost:8000
localhost:8000 retirada
sending test consensus to localhost:8001
localhost:8001 atacar
sending test consensus to localhost:8002
localhost:8003 retirada
localhost:8002 atacar
sending test consensus to localhost:8003
sending test consensus to localhost:8004
sending test consensus to localhost:8005
sending test consensus to localhost:8006
localhost:8004 atacar
localhost:8005 atacar
localhost:8006 retirada
localhost:8003 ATACAR!!!!
localhost:8003 time to die
localhost:8005 ATACAR!!!!
localhost:8005 time to die
localhost:8001 ATACAR!!!!
localhost:8001 time to die
localhost:8002 ATACAR!!!!
localhost:8002 time to die
localhost:8000 ATACAR!!!!
localhost:8000 time to die
localhost:8004 ATACAR!!!!
localhost:8004 time to die
localhost:8006 ATACAR!!!!
localhost:8006 time to die


In [ ]:
%%script bash
#kill -9 2607

ps ax

    PID TTY      STAT   TIME COMMAND
      1 ?        Ss     0:00 /bin/bash -e /datalab/run.sh
      8 ?        Sl     0:01 /tools/node/bin/node /datalab/web/app.js
     18 ?        Sl     0:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
    113 ?        Ss     0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
    121 ?        Ssl    0:02 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-4a26edd2-bb76-4d94-a7a5-d57c443e75c7.json
   1308 ?        S      0:00 bash
   1309 ?        R      0:00 ps ax
